In [ ]:
%cd ..

In [ ]:
import json
import numpy as np
import pandas as pd
from notebooks.utils import *
pd.set_option('display.max_rows', 500)

In [ ]:
TEX_TABLE_DIR = "../masters-thesis-tex/tables/"

MODEL_COL = "model"
DATASET_COL = "dataset"
IMAGE_COL = "image_index"
NOISE_COL = "noise_stddev"
METRIC_COL = "metric"
VALUE_COL = "value"

DATASETS_MAPPING = {
    "CBSD68": "CBSD68 \cite{martin_database_2001}",
    "Kodak24": "Kodak24 \cite{franzen_kodak_2021}",
    "McMaster": "McMaster \cite{wu_color_2011}",
}

In [ ]:
# denoising_eval_csv = 'results/denoising/models/all.csv'  # Result of /scripts/eval_denoising_all.py
# denoising_eval_cbm3d_csv = 'results/denoising/cbm3d.csv'  # Result of /other-methods/denoising/cbm3d/main.py
# denoising_eval_dncnn_csv = 'results/denoising/dncnn.csv'  # Result of /other-methods/denoising/dncnn/main.py

# # Read the evaluation
# df_models = pd.read_csv(denoising_eval_csv)
# df_cbm3d = pd.read_csv(denoising_eval_cbm3d_csv)
# df_dncnn = pd.read_csv(denoising_eval_dncnn_csv)

# # Combine the datasets
# df_cbm3d[MODEL_COL] = "CBM3D"
# df_dncnn[MODEL_COL] = "CDnCNN-B"
# df_all = pd.concat([df_models, df_cbm3d, df_dncnn], axis=0)
df_all = load_folder('results/denoising/', MODEL_COL)


In [ ]:
# Read the model name mapping
with open('models/model_name_mapping.json') as f:
    model_mapping = json.load(f)

model_mapping = {
    "cbm3d": "CBM3D \cite{makinen_collaborative_2020}",
    "dncnn": "CDnCNN-B \cite{zhang_beyond_2017}",
    **model_mapping
}
models = list(model_mapping.keys())

In [ ]:
print(df_all[DATASET_COL].unique())

# Main Text Table

In [ ]:
dataset = "CBSD68"
metrics = ["PSNR", "LPIPS_ALEX"]
metrics_mapping = {
    "PSNR": "PSNR",
    "LPIPS_ALEX": "LPIPS",
}
noise_levels = [0.05, 0.10, 0.20]

In [ ]:
df = df_all.copy()

# Filter only the configured dataset
df = df[df[DATASET_COL] == dataset]
df = df.drop(DATASET_COL, axis=1)

# Mean over the images
df = df.groupby([MODEL_COL, NOISE_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[MODEL_COL, NOISE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[MODEL_COL].isin(models)]
df = df[df[NOISE_COL].isin(noise_levels)]

# Sort by Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(NOISE_COL, kind="mergesort")

# Remap metrics
df[METRIC_COL] = df[METRIC_COL].map(metrics_mapping)

# Noise Level and Metric as columns
df = df.set_index([MODEL_COL, NOISE_COL, METRIC_COL])
df = df.unstack([NOISE_COL, METRIC_COL])

# Sort and remap models
df = df.reset_index()
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)
df = df.set_index(MODEL_COL)

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["Additive noise \( \sigma_n \)", ""])

# Rename model column
df.index = df.index.rename("Model")

df


## Convert to LaTeX

In [ ]:
column_format = "l" + "C{1}" * len(df.columns)


formatters = {
    c: mark_column_best_formatter(
        df, c, mark_max=c[1] == "PSNR", num_decimals=2 if c[1] == "PSNR" else 4
    )
    for c in df.columns
}

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        # float_format=float_format(True),
        formatters=formatters,
        escape=False,
        column_format=column_format,
        multicolumn_format="c",
        multirow=True,
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

latex = add_midrule(latex, 10)
latex = add_midrule(latex, 18)

print(latex)


In [ ]:
with open(TEX_TABLE_DIR + "denoising_cbsd68.tex", "w") as f:
    f.write(latex)

# Appendix Table

In [ ]:
datasets = ["CBSD68", "Kodak24", "McMaster"]
metrics = ["PSNR", "SSIM", "FSIM", "LPIPS_ALEX"]
metrics_mapping = {
    "PSNR": "P",
    "SSIM": "S",
    "FSIM": "F",
    "LPIPS_ALEX": "L",
}
noise_levels = [0.01, 0.05, 0.10, 0.20]

In [ ]:
df = df_all.copy()

# Mean over the images
df = df.groupby([MODEL_COL, NOISE_COL, DATASET_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[MODEL_COL, DATASET_COL, NOISE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[DATASET_COL].isin(datasets)]
df = df[df[MODEL_COL].isin(models)]

# Remap names
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)

# Models as columns
df = df.set_index([DATASET_COL, NOISE_COL, METRIC_COL, MODEL_COL])
df = df.unstack(MODEL_COL)

df = df.reset_index()

# Sort by Dataset, Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(NOISE_COL, kind="mergesort")
df = df.sort_values(DATASET_COL, key=sort_key_for(datasets), kind="mergesort")

# Rename Dataset, Noise, and Metric
df[DATASET_COL] = df[DATASET_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{" + DATASETS_MAPPING[x] + "}"
)
df[NOISE_COL] = df[NOISE_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{\( \sigma_n \)=" + str(x) + " }"
)
df[METRIC_COL] = df[METRIC_COL].map(metrics_mapping)

df = df.set_index([DATASET_COL, NOISE_COL, METRIC_COL])

# Update the column index and index
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename("")
df.index = df.index.rename(["", "", ""])

# Sort models
df = df.reindex(list(model_mapping.values()), axis=1)

# Replace nan
df = df.replace(np.nan, "---")

df


## Convert to LaTeX

In [ ]:
column_format = "lll|" + "C{0.7}" * 4 + "|" + "C{1}" * (len(df.columns) - 4)

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        float_format=float_format(True),
        escape=False,
        na_rep="---",
        column_format=column_format,
        multicolumn_format="c",
        multirow=True,
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

latex = delete_line(latex, 3)

# print(latex)

## Save to tex file

In [ ]:
with open(TEX_TABLE_DIR + "all_denoising.tex", "w") as f:
    f.write(latex)
